## With Chroma Local


In [1]:
import gradio as gr
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

from llama_index.core import Settings, VectorStoreIndex, StorageContext
from chromadb.config import Settings as StChroma


from llama_index.llms.openai import OpenAI

# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
import dotenv


/home/isaiaszc/inter_projects/personal-RAG/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configure the environment

dotenv.load_dotenv()

llm = OpenAI(model="gpt-4o-mini", temperature=0.2)

embedding = OpenAIEmbedding(model="text-embedding-3-small")

Settings.llm = llm
Settings.embed_model = embedding


In [3]:
# Chroma Cliente from local
chroma_client = chromadb.PersistentClient(
    path="./chroma_recipes",
    settings=StChroma(allow_reset=True),
)

chroma_collection = chroma_client.get_or_create_collection(name="recipes")


In [4]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    storage_context=storage_context,
)


query_engine = index.as_query_engine()


In [11]:
promp = query_engine.get_prompts()

# query_engine.update_prompts(promp)


In [13]:
from llama_index.core.prompts import BasePromptTemplate, PromptTemplate

PROMPT_MODEL = """
The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 
"""


AttributeError: 'RetrieverQueryEngine' object has no attribute 'get_refine_template'

In [12]:
print(promp)


{'response_synthesizer:text_qa_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '), conditionals=[(<function is_chat_model at 0x7f8ce66b2660>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.

## Conversation with Gradio


In [8]:
def conversation(query, history=None):
    response = query_engine.query(query)
    return response.response


demo = gr.ChatInterface(conversation)


In [9]:
demo.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
